##  O objetivo / The objective

- Com o dataset das seleções mundiais, e os dados contidos nele, modelar a distribuição de gols de cada time em uma partida, e descobrir a media de gols em cada partida

--------------------------------------------------------------------------------------------------------------

- With the dataset of the world teams, and the data contained therein, model the distribution of goals for each team in a match, and discover the average goals in each match

In [1]:
# Importando as bibliotecas
# Import the libraries
import pandas as pd
import numpy as np
from scipy.stats import poisson

In [2]:
teams = pd.read_excel("DadosCopaDoMundoQatar2022.xlsx", sheet_name='selecoes', index_col=0)
match = pd.read_excel("DadosCopaDoMundoQatar2022.xlsx", sheet_name='jogos')

In [3]:
teams.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [4]:
match.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [5]:
# Definindo as forças do time
# Defining team strengths
fifa = teams['PontosRankingFIFA']
a, b = min(fifa), max(fifa)
# definindo os intervalos que eu quero das seleções
# defining the ranges I want from the selections
fa, fb = 0.15, 1
# transformação linear de escala numerica
# Numerical scale linear transformation
b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
strenght = b0 + b1*fifa
strenght

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [6]:
# Usando a metrica POISSON para calcular as probabilidades
# Using the POISSON metric to calculate the probability
def PoissonMean(team1, team2):
    strenght1 = strenght[team1]
    strenght2 = strenght[team2]
    goalsm= 2.75
    l1 = goalsm * strenght1/(strenght1 + strenght2)
    l2 = goalsm - l1
    return [l1, l2]

In [7]:
# Função para definir vitoria, empate e derrota
# Function to define victory, draw and defeat
def Result(goals1, goals2):
    if goals1 > goals2:
        res = 'Victory'
    if goals1 < goals2:
        res = 'Defeat' 
    if goals1 == goals2:
        res = 'Draw'       
    return res

In [8]:
# Função de sistema de pontos na tabela apos a predição da partida para Vitoria empate e derrota
# Function of table system points after the match prediction to victory, draw and defeat  
def Points(goals1, goals2):
    rst = Result(goals1, goals2)
    if rst == 'Victory':
        points1, points2 = 3, 0
    if rst == 'Draw':
        points1, points2 = 1, 1
    if rst == 'Defeat':
        points1, points2 = 0, 3
    return points1, points2, rst

In [9]:
# Função para retornar o valor do placar do jogo(saldo de gols)
# Function to return the match scoreboard values, (goals difference)
def Match(team1, team2):
    l1, l2 = PoissonMean(team1, team2) 
    goals1 = int(np.random.poisson(lam = l1, size = 1))
    goals2 = int(np.random.poisson(lam = l2, size = 1))
    saldo1 = goals1 - goals2
    saldo2 = goals2 - goals1 # Ou -saldo1
    points1, points2, result = Points(goals1, goals2)
    placar ='{}X{}'.format(goals1, goals2)
    return [goals1, goals2, saldo1, saldo2, points1, points2, result, placar]

In [10]:
# Calcula a probabilidade dos times de marcar gols com a metrica POISSON
# Calculate the probability of the teams score with POISSON metrics
def Distribution(media):
    probs = []
    for i in range(7):
        probs.append(poisson.pmf(i, media))
    probs.append(1 - sum(probs))
    return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])

In [11]:
# Função de calculo das probabilidades de gols, vitoria, empate e derrota e pontos das duas seleções
# Function to calculate the goals probabilities, victory, draw and defeat and the the points of the teams
def MatchProbability(team1, team2):
    l1, l2 = PoissonMean(team1, team2)
    d1, d2 = Distribution(l1), Distribution(l2)
    matrix = np.outer(d1, d2)
    victory = np.tril(matrix).sum()-np.trace(matrix) #Soma o triangulo inferior
    defeat = np.triu(matrix).sum()-np.trace(matrix) #Soma o triangulo inferior
    draw = 1 - (victory + defeat)
    
    probs = np.around([victory, draw, defeat], 3)
    probsp = [f'{100*i:.1f}%' for i in probs]
    
    names = ['0','1','2','3','4','5','6','7+']
    matrix = pd.DataFrame(matrix, columns = names, index = names)
    matrix.index = pd.MultiIndex.from_product([[team1], matrix.index])
    matrix.columns = pd.MultiIndex.from_product([[team2], matrix.columns])
    
    output = {'team1': team1, 'team2': team2,
              'f1': strenght[team1], 'f2': strenght[team2],
              'media1': l1, 'media2': l2,
              'probabilities': probsp, 'matrix': matrix}
    
    return output

In [12]:
MatchProbability('Brasil', 'França')

{'team1': 'Brasil',
 'team2': 'França',
 'f1': 1.0,
 'f2': 0.8605617977528088,
 'media1': 1.478048191315901,
 'media2': 1.271951808684099,
 'probabilities': ['42.0%', '25.4%', '32.6%'],
 'matrix':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
                   6     

In [13]:
# Prevendo as partidas
# Predicting matches
match['Victory'] = None
match['Draw'] = None
match['Defeat'] = None

for i in range(match.shape[0]):
    team1 = match['seleção1'][i]
    team2 = match['seleção2'][i]
    vi, dr, de = MatchProbability(team1, team2)['probabilities']
    match.at[i,'Victory'] = vi
    match.at[i,'Draw'] = dr
    match.at[i,'Defeat'] = de
match

,data,hora,grupo,seleção1,seleção2,estádio,Victory,Draw,Defeat
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


In [14]:
# Salvando em um arquivo Excel
# Save to Excel file
match.to_excel('OutputWorldCupGameEstimates.xlsx')